In [49]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, datasets
from torchvision.transforms import v2
import transformers
from transformers import AutoImageProcessor, AutoModelForImageClassification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
from tqdm import tqdm
import time
import json
import warnings
warnings.filterwarnings('ignore')

In [50]:
# Definição de constantes
IMAGE_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 50
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5
NUM_CLASSES = 2  # ALL e HEM
EARLY_STOPPING_PATIENCE = 10  # Número de épocas para aguardar melhoria
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {DEVICE}")

# Diretórios
train_dir = 'dataset/processado/train/'
val_dir = 'dataset/processado/validation/'
test_dir = 'dataset/processado/test/'
save_dir = 'models/benchmark_facebook'
results_dir = 'results/benchmark_facebook'
os.makedirs(save_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)
os.makedirs(os.path.join(results_dir, 'plots'), exist_ok=True)

Usando dispositivo: cuda


In [51]:
# Modelos a serem avaliados
models_config = {
    'convnext-base': 'facebook/convnext-base-224',
    'ViT-Base' :'google/vit-base-patch16-224',
    'Swin-Base' : 'microsoft/swin-base-patch4-window7-224'
}

In [52]:
# Classe para implementar Early Stopping
class EarlyStopping:
    """Early Stopping para interromper o treinamento quando a validação parar de melhorar."""
    def __init__(self, patience=10, min_delta=0, verbose=True):
        """
        Args:
            patience (int): Quantas épocas esperar após a última melhoria.
            min_delta (float): Mínima mudança para considerar como melhoria.
            verbose (bool): Se deve imprimir mensagem quando ocorrer early stopping.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        
    def __call__(self, val_loss, model, model_path):
        score = -val_loss
        
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, model_path)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping: {self.counter}/{self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print('EarlyStopping: Parando treinamento')
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, model_path)
            self.counter = 0
            
    def save_checkpoint(self, val_loss, model, model_path):
        """Salva o modelo quando a validação melhora."""
        if self.verbose:
            print(f'Validação melhorou ({self.val_loss_min:.6f} --> {val_loss:.6f}). Salvando modelo...')
        torch.save(model.state_dict(), model_path)
        self.val_loss_min = val_loss

In [53]:
# Definição das transformações de dados
def get_transforms(processor):
    """Obtém as transformações de pré-processamento para cada modelo"""
    # Extrair médias e desvios padrão seguros
    if hasattr(processor, 'image_mean') and hasattr(processor, 'image_std'):
        mean = processor.image_mean
        std = processor.image_std
    else:
        # Valores padrão do ImageNet caso não estejam disponíveis
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        print(f"AVISO: Usando valores padrão do ImageNet para mean e std.")
    
    train_transforms = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(20),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ])
    
    val_transforms = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ])
    
    return train_transforms, val_transforms

In [54]:
# Função para carregar os datasets
def load_datasets(train_transforms, val_transforms):
    """Carrega os datasets de treino, validação e teste"""
    train_dataset = datasets.ImageFolder(
        root=train_dir,
        transform=train_transforms
    )
    
    val_dataset = datasets.ImageFolder(
        root=val_dir,
        transform=val_transforms
    )
    
    test_dataset = datasets.ImageFolder(
        root=test_dir,
        transform=val_transforms
    )
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=True, 
        num_workers=4,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False, 
        num_workers=4,
        pin_memory=True
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False, 
        num_workers=4,
        pin_memory=True
    )
    
    class_names = train_dataset.classes
    
    dataset_info = {
        'train_size': len(train_dataset),
        'val_size': len(val_dataset),
        'test_size': len(test_dataset),
        'class_names': class_names,
        'class_distribution': {
            'train': {
                class_names[0]: sum([1 for _, label in train_dataset if label == 0]),
                class_names[1]: sum([1 for _, label in train_dataset if label == 1])
            },
            'val': {
                class_names[0]: sum([1 for _, label in val_dataset if label == 0]),
                class_names[1]: sum([1 for _, label in val_dataset if label == 1])
            },
            'test': {
                class_names[0]: sum([1 for _, label in test_dataset if label == 0]),
                class_names[1]: sum([1 for _, label in test_dataset if label == 1])
            }
        }
    }
    
    print(f"Dataset carregado:")
    print(f"Treino: {len(train_dataset)} imagens")
    print(f"Validação: {len(val_dataset)} imagens")
    print(f"Teste: {len(test_dataset)} imagens")
    print(f"Classes: {class_names}")
    
    return train_loader, val_loader, test_loader, dataset_info


In [55]:
# Função para treinar o modelo
def train_model(model, train_loader, val_loader, optimizer, criterion, scheduler, model_name, epochs=EPOCHS):
    """Treina o modelo e retorna histórico de métricas"""
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'val_precision': [],
        'val_recall': [],
        'val_f1': [],
        'time_per_epoch': []
    }
    
    best_val_loss = float('inf')
    best_model_path = os.path.join(save_dir, f"{model_name}_best.pth")
    
    # Early Stopping: parâmetros
    patience = 10  # Número de épocas para esperar por melhoria
    min_delta = 0.0001  # Mínima mudança considerada como melhoria
    early_stop_counter = 0
    early_stopped = False
    best_epoch = 0
    
    for epoch in range(epochs):
        start_time = time.time()
        
        # Modo de treinamento
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_loop = tqdm(train_loader, desc=f"Época {epoch+1}/{epochs} (Treino)")
        for batch_idx, (inputs, targets) in enumerate(train_loop):
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(inputs).logits
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += targets.size(0)
            train_correct += predicted.eq(targets).sum().item()
            
            # Atualiza a barra de progresso sem prints excessivos
            if batch_idx % 10 == 0:  # Atualizar a cada 10 batches
                train_loop.set_postfix(
                    loss=train_loss/(batch_idx+1),
                    acc=100.*train_correct/train_total
                )
        
        train_loss = train_loss / len(train_loader)
        train_acc = 100. * train_correct / train_total
        
        # Modo de validação
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_targets = []
        
        with torch.no_grad():
            val_loop = tqdm(val_loader, desc=f"Época {epoch+1}/{epochs} (Val)")
            for batch_idx, (inputs, targets) in enumerate(val_loop):
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                
                outputs = model(inputs).logits
                loss = criterion(outputs, targets)
                
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                
                # Acumular apenas os índices ou classes ao invés dos tensores completos
                val_preds.extend(predicted.cpu().numpy())
                val_targets.extend(targets.cpu().numpy())
                
                # Atualiza a barra de progresso com menos frequência
                if batch_idx % 10 == 0:  # Atualizar a cada 10 batches
                    val_loop.set_postfix(loss=val_loss/(batch_idx+1))
        
        val_loss = val_loss / len(val_loader)
        
        # Calcular métricas apenas uma vez, no final da época
        val_acc = 100. * accuracy_score(val_targets, val_preds)
        val_precision = precision_score(val_targets, val_preds, average='weighted')
        val_recall = recall_score(val_targets, val_preds, average='weighted')
        val_f1 = f1_score(val_targets, val_preds, average='weighted')
        
        # Ajuste do learning rate
        if scheduler:
            scheduler.step(val_loss)
            current_lr = optimizer.param_groups[0]['lr']
            print(f"Learning rate atual: {current_lr:.6f}")
        
        # Tempo gasto na época
        epoch_time = time.time() - start_time
        
        # Salvar métricas
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_precision'].append(val_precision)
        history['val_recall'].append(val_recall)
        history['val_f1'].append(val_f1)
        history['time_per_epoch'].append(epoch_time)
        
        # Imprimir resultados
        print(f"Época {epoch+1}/{epochs} - "
              f"Tempo: {epoch_time:.2f}s - "
              f"Treino Loss: {train_loss:.4f} - "
              f"Treino Acc: {train_acc:.2f}% - "
              f"Val Loss: {val_loss:.4f} - "
              f"Val Acc: {val_acc:.2f}% - "
              f"Val F1: {val_f1:.4f}")
        
        # Early Stopping: verificar se houve melhoria
        if val_loss < best_val_loss - min_delta:
            # Houve melhoria significativa
            best_val_loss = val_loss
            best_epoch = epoch
            early_stop_counter = 0
            
            # Salvar o melhor modelo
            torch.save(model.state_dict(), best_model_path)
            print(f"Melhor modelo salvo com Val Loss: {val_loss:.4f}")
        else:
            # Não houve melhoria significativa
            early_stop_counter += 1
            
            # Reduzir frequência de logs
            if early_stop_counter == 1 or early_stop_counter % 2 == 0 or early_stop_counter == patience:
                print(f"EarlyStopping: {early_stop_counter}/{patience}")
            
            # Verificar se devemos parar o treinamento
            if early_stop_counter >= patience:
                print(f"Early Stopping ativado na época {epoch+1}. Treinamento interrompido.")
                early_stopped = True
                break
                
        # Liberar memória explicitamente
        torch.cuda.empty_cache()
    
    # Carregar o melhor modelo para avaliação
    model.load_state_dict(torch.load(best_model_path))
    
    # Adicionar informação de early stopping ao histórico
    if early_stopped:
        print(f"Treinamento interrompido por Early Stopping após {epoch+1} épocas.")
        print(f"Usando o melhor modelo da época {best_epoch+1} com val_loss: {best_val_loss:.4f}")
    else:
        print(f"Treinamento concluído sem Early Stopping. Melhor val_loss: {best_val_loss:.4f} (época {best_epoch+1})")
    
    return model, history

In [56]:
# Função para avaliar o modelo no conjunto de teste
def evaluate_model(model, test_loader, class_names):
    """Avalia o modelo no conjunto de teste e retorna métricas"""
    model.eval()
    
    # Inicialização de listas para armazenar previsões e alvos
    all_preds = []
    all_targets = []
    all_probs = []
    
    with torch.no_grad():
        for inputs, targets in tqdm(test_loader, desc="Avaliando no conjunto de teste"):
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            
            outputs = model(inputs).logits
            probs = torch.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    # Cálculo das métricas
    accuracy = accuracy_score(all_targets, all_preds)
    precision = precision_score(all_targets, all_preds, average='weighted')
    recall = recall_score(all_targets, all_preds, average='weighted')
    f1 = f1_score(all_targets, all_preds, average='weighted')
    
    # Matriz de confusão
    cm = confusion_matrix(all_targets, all_preds)
    
    # Cálculo da curva ROC para classificação binária
    fpr, tpr, _ = roc_curve(all_targets, np.array(all_probs)[:, 1])
    roc_auc = auc(fpr, tpr)
    
    # Resultados
    results = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm.tolist(),
        'roc_auc': roc_auc,
        'fpr': fpr.tolist(),
        'tpr': tpr.tolist()
    }
    
    print(f"Métricas de Teste:")
    print(f"Acurácia: {accuracy:.4f}")
    print(f"Precisão: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"AUC-ROC: {roc_auc:.4f}")
    print("Matriz de Confusão:")
    print(cm)
    
    return results

In [57]:
# Função para plotar a matriz de confusão
def plot_confusion_matrix(cm, class_names, model_name):
    """Plota a matriz de confusão"""
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predição')
    plt.ylabel('Real')
    plt.title(f'Matriz de Confusão - {model_name}')
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, 'plots', f'{model_name}_confusion_matrix.png'), dpi=300)
    plt.close()

In [58]:
# Função para plotar a curva ROC
def plot_roc_curve(fpr, tpr, roc_auc, model_name):
    """Plota a curva ROC"""
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taxa de Falsos Positivos')
    plt.ylabel('Taxa de Verdadeiros Positivos')
    plt.title(f'Curva ROC - {model_name}')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, 'plots', f'{model_name}_roc_curve.png'), dpi=300)
    plt.close()

In [59]:
# Função para plotar o histórico de treinamento
def plot_training_history(history, model_name):
    """Plota o histórico de treinamento"""
    # Plot de acurácia
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_acc'], label='Treino')
    plt.plot(history['val_acc'], label='Validação')
    plt.title(f'Acurácia - {model_name}')
    plt.xlabel('Época')
    plt.ylabel('Acurácia (%)')
    plt.legend()
    plt.grid(True)
    
    # Plot de perda
    plt.subplot(1, 2, 2)
    plt.plot(history['train_loss'], label='Treino')
    plt.plot(history['val_loss'], label='Validação')
    plt.title(f'Perda - {model_name}')
    plt.xlabel('Época')
    plt.ylabel('Perda')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, 'plots', f'{model_name}_training_history.png'), dpi=300)
    plt.close()
    
    # Plot de F1, Precisão e Recall
    plt.figure(figsize=(10, 6))
    plt.plot(history['val_precision'], label='Precisão')
    plt.plot(history['val_recall'], label='Recall')
    plt.plot(history['val_f1'], label='F1-Score')
    plt.title(f'Métricas de Validação - {model_name}')
    plt.xlabel('Época')
    plt.ylabel('Valor')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, 'plots', f'{model_name}_validation_metrics.png'), dpi=300)
    plt.close()

In [60]:
# Função principal para o benchmarking
def run_benchmark():
    """Executa o benchmark de todos os modelos"""
    all_results = {}
    benchmark_start_time = time.time()
    
    # Mostrar informações do sistema e versões das bibliotecas
    print("\n=== Informações do Sistema ===")
    print(f"Python: {sys.version}")
    print(f"PyTorch: {torch.__version__}")
    print(f"Torchvision: {torchvision.__version__}")
    print(f"Transformers: {transformers.__version__}")
    print(f"Dispositivo: {DEVICE}")
    print(f"GPU disponível: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
    print("=============================\n")
    
    # Iterar sobre cada modelo
    for model_name, model_id in models_config.items():
        print(f"\n{'='*50}")
        print(f"Benchmark do modelo: {model_name}")
        print(f"{'='*50}")
        
        try:
            # Carregar o processador de imagem e o modelo
            processor = AutoImageProcessor.from_pretrained(model_id)
            model = AutoModelForImageClassification.from_pretrained(
                model_id,
                num_labels=NUM_CLASSES,
                ignore_mismatched_sizes=True
            )
            model = model.to(DEVICE)
            
            # Configurar transformações e datasets
            train_transforms, val_transforms = get_transforms(processor)
            train_loader, val_loader, test_loader, dataset_info = load_datasets(train_transforms, val_transforms)
            
            # Configurar otimizador e critério
            optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
            criterion = nn.CrossEntropyLoss()
            # Código corrigido sem o parâmetro 'verbose'
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
            print(f"Configurado scheduler ReduceLROnPlateau com patience=5, factor=0.5")
            
            # Treinar o modelo
            model_start_time = time.time()
            print(f"Iniciando treinamento do modelo {model_name}...")
            model, history = train_model(model, train_loader, val_loader, optimizer, criterion, scheduler, model_name)
            
            # Avaliar no conjunto de teste
            print(f"Avaliando modelo {model_name} no conjunto de teste...")
            test_results = evaluate_model(model, test_loader, dataset_info['class_names'])
            
            # Calcular tempo total
            model_total_time = time.time() - model_start_time
            
            # Salvar resultados
            all_results[model_name] = {
                'test_metrics': test_results,
                'training_history': history,
                'training_time': model_total_time
            }
            
            # Salvar o modelo final
            final_model_path = os.path.join(save_dir, f"{model_name}_final.pth")
            torch.save(model.state_dict(), final_model_path)
            
            # Plotar resultados
            plot_confusion_matrix(np.array(test_results['confusion_matrix']), dataset_info['class_names'], model_name)
            plot_roc_curve(test_results['fpr'], test_results['tpr'], test_results['roc_auc'], model_name)
            plot_training_history(history, model_name)
            
            # Salvar informações do modelo
            with open(os.path.join(results_dir, f'{model_name}_results.json'), 'w') as f:
                json.dump({
                    'model_name': model_name,
                    'model_id': model_id,
                    'test_metrics': {
                        'accuracy': test_results['accuracy'],
                        'precision': test_results['precision'],
                        'recall': test_results['recall'],
                        'f1_score': test_results['f1_score'],
                        'roc_auc': test_results['roc_auc']
                    },
                    'training_time': model_total_time,
                    'epochs_trained': len(history['train_loss']),
                    'dataset_info': dataset_info
                }, f, indent=4)
            
            print(f"Benchmark do modelo {model_name} concluído em {model_total_time/60:.2f} minutos")
            
        except Exception as e:
            print(f"Erro ao avaliar o modelo {model_name}: {e}")
            all_results[model_name] = {'error': str(e)}
    
    # Salvar resultados comparativos
    comparison_results = {}
    for model_name, result in all_results.items():
        if 'error' not in result:
            comparison_results[model_name] = {
                'accuracy': result['test_metrics']['accuracy'],
                'precision': result['test_metrics']['precision'],
                'recall': result['test_metrics']['recall'],
                'f1_score': result['test_metrics']['f1_score'],
                'roc_auc': result['test_metrics']['roc_auc'],
                'training_time': result['training_time'] / 60  # em minutos
            }
    
    # Criar DataFrame para comparação
    df_comparison = pd.DataFrame(comparison_results).T
    df_comparison = df_comparison.sort_values(by='f1_score', ascending=False)
    
    # Salvar comparação em CSV
    df_comparison.to_csv(os.path.join(results_dir, 'model_comparison.csv'))
    
    # Plotar comparação de acurácia e F1-score
    plt.figure(figsize=(15, 10))
    
    # Acurácia
    ax1 = plt.subplot(2, 1, 1)
    df_comparison['accuracy'].plot(kind='bar', ax=ax1, color='skyblue')
    plt.title('Comparação de Acurácia entre Modelos')
    plt.ylabel('Acurácia')
    plt.ylim([0, 1])
    plt.grid(axis='y')
    plt.xticks(rotation=45)
    
    for i, v in enumerate(df_comparison['accuracy']):
        plt.text(i, v + 0.01, f"{v:.4f}", ha='center', fontsize=9)
    
    # F1-Score
    ax2 = plt.subplot(2, 1, 2)
    df_comparison['f1_score'].plot(kind='bar', ax=ax2, color='lightgreen')
    plt.title('Comparação de F1-Score entre Modelos')
    plt.ylabel('F1-Score')
    plt.ylim([0, 1])
    plt.grid(axis='y')
    plt.xticks(rotation=45)
    
    for i, v in enumerate(df_comparison['f1_score']):
        plt.text(i, v + 0.01, f"{v:.4f}", ha='center', fontsize=9)
    
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, 'plots', 'model_comparison_metrics.png'), dpi=300)
    plt.close()
    
    # Comparação de tempo de treinamento
    plt.figure(figsize=(15, 6))
    df_comparison['training_time'].plot(kind='bar', color='salmon')
    plt.title('Comparação de Tempo de Treinamento entre Modelos')
    plt.ylabel('Tempo (minutos)')
    plt.grid(axis='y')
    plt.xticks(rotation=45)
    
    for i, v in enumerate(df_comparison['training_time']):
        plt.text(i, v + 0.5, f"{v:.1f}", ha='center', fontsize=9)
    
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, 'plots', 'model_comparison_time.png'), dpi=300)
    plt.close()
    
    # Criar gráfico de radar para comparar todos os modelos
    metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']
    
    # Preparar dados para o gráfico de radar
    plt.figure(figsize=(12, 10))
    
    # Número de variáveis
    N = len(metrics)
    
    # Ângulos para cada eixo
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]
    
    # Inicializar o gráfico de radar
    ax = plt.subplot(111, polar=True)
    
    # Definir os ângulos
    plt.xticks(angles[:-1], metrics, size=12)
    
    # Desenhar o limite y
    # Definir os ângulos
    plt.xticks(angles[:-1], metrics, size=10)

    # Desenhar o limite y
    ax.set_rlabel_position(0)
    plt.yticks([0.25, 0.5, 0.75, 1.0], ["0.25", "0.50", "0.75", "1.00"], color="grey", size=8)
    plt.ylim(0, 1)
    
    # Plotar cada modelo
    colors = plt.cm.tab10(np.linspace(0, 1, len(df_comparison)))
    
    for i, (model_name, model_metrics) in enumerate(df_comparison.iterrows()):
        values = model_metrics[metrics].values.flatten().tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=model_name, color=colors[i])
        ax.fill(angles, values, alpha=0.1, color=colors[i])
    
    # Adicionar legenda
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.title('Comparação de Métricas entre Modelos', size=15, y=1.1)
    
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, 'plots', 'model_comparison_radar.png'), dpi=300)
    plt.close()
    
    # Cálculo do tempo total de benchmark
    benchmark_total_time = time.time() - benchmark_start_time
    print(f"\nBenchmark completo concluído em {benchmark_total_time/60:.2f} minutos")
    print(f"Resultados salvos em: {results_dir}")
    
    # Retornar todos os resultados
    return all_results, df_comparison

if __name__ == "__main__":
    print(f"Iniciando benchmark em {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    run_benchmark()
    print(f"Benchmark concluído em {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Iniciando benchmark em 2025-05-15 22:31:27

=== Informações do Sistema ===
Python: 3.10.12 (main, Feb  4 2025, 14:57:36) [GCC 11.4.0]
PyTorch: 2.7.0+cu126
Torchvision: 0.22.0+cu126
Transformers: 4.51.3
Dispositivo: cuda
GPU disponível: True
GPU: NVIDIA GeForce RTX 4060


Benchmark do modelo: convnext-base


Some weights of ConvNextForImageClassification were not initialized from the model checkpoint at facebook/convnext-base-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([2, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Dataset carregado:
Treino: 16384 imagens
Validação: 2326 imagens
Teste: 976 imagens
Classes: ['all', 'hem']
Configurado scheduler ReduceLROnPlateau com patience=5, factor=0.5
Iniciando treinamento do modelo convnext-base...


Época 1/50 (Val): 100%|██████████| 146/146 [00:18<00:00,  8.09it/s, loss=0.164]


Learning rate atual: 0.000100
Época 1/50 - Tempo: 424.03s - Treino Loss: 0.3827 - Treino Acc: 82.23% - Val Loss: 0.1937 - Val Acc: 92.73% - Val F1: 0.9250
Melhor modelo salvo com Val Loss: 0.1937


...

Learning rate atual: 0.000025
Época 30/50 - Tempo: 373.75s - Treino Loss: 0.0236 - Treino Acc: 99.13% - Val Loss: 0.1196 - Val Acc: 96.82% - Val F1: 0.9684


Época 31/50 (Val): 100%|██████████| 146/146 [00:16<00:00,  8.60it/s, loss=0.201]  


Learning rate atual: 0.000025
Época 31/50 - Tempo: 384.16s - Treino Loss: 0.0236 - Treino Acc: 99.14% - Val Loss: 0.1995 - Val Acc: 94.41% - Val F1: 0.9455
EarlyStopping: 10/10
Early Stopping ativado na época 31. Treinamento interrompido.
Treinamento interrompido por Early Stopping após 31 épocas.
Usando o melhor modelo da época 21 com val_loss: 0.0679
Avaliando modelo Swin-Base no conjunto de teste...


Avaliando no conjunto de teste: 100%|██████████| 61/61 [00:08<00:00,  7.56it/s]


Métricas de Teste:
Acurácia: 0.9201
Precisão: 0.9192
Recall: 0.9201
F1-Score: 0.9171
AUC-ROC: 0.9454
Matriz de Confusão:
[[731  18]
 [ 60 167]]
Benchmark do modelo Swin-Base concluído em 195.18 minutos

Benchmark completo concluído em 608.70 minutos
Resultados salvos em: results/benchmark_facebook
Benchmark concluído em 2025-05-16 08:40:09
